In [16]:
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Activation, Dropout, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


In [17]:
train_images  = pd.read_csv("../input/hand-gesture-recognition-2nd-edition/train.csv")

In [18]:
train_labels  = train_images.iloc[:,1]
train_images  = train_images.iloc[:,2:]


In [19]:
train_labels  = train_labels.to_numpy()
train_images = train_images.to_numpy()
training_images=train_images.reshape(train_images.shape[0], 211, 271, 1)
train_images=training_images / 255.0

In [20]:
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(training_images[i], cmap = "gray")
    plt.title(train_labels[i])
    plt.axis("off")

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_images, train_labels, test_size = 0.2, random_state = 42)

In [22]:
train_datagen = ImageDataGenerator(
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    fill_mode='nearest'
)

In [23]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(211, 271, 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax')
])


In [24]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [25]:
history = model.fit(train_datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32,
                    epochs=5,
                    validation_data=validation_datagen.flow(X_valid, y_valid, batch_size=32))

In [26]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_df = pd.read_csv("/kaggle/input/hand-gesture-recognition-2nd-edition/test.csv")
submission = pd.read_csv("/kaggle/input/hand-gesture-recognition-2nd-edition/sample_submission.csv")
X_test = test_df.drop('id', axis=1).values
X_test = X_test.reshape((len(X_test),211, 271,1))

predictions = model.predict(X_test).argmax(axis=1)

submission['label'] = predictions
submission.to_csv('submission.csv', index=False)